# Explaining Model Decisions and Performance

## Overview

At a very high level, Howso Engine is about: 

- Making an accurate prediction (even with limited or sparse data!) 

- Explaining the prediction process 

- Showing key properties of the data 

In this notebook, we will be using the adult data set as an example to demonstrate some of Howso Engine’s capabilities, including cases and features which contribute to predictions, anomalies analysis, and potential improvements to the data to gain more insight into the data.  


In [1]:
import pandas as pd
from pmlb import fetch_data

from howso import engine
from howso.utilities import infer_feature_attributes
from howso.visuals import (
    plot_anomalies,
    plot_dataset,
    plot_feature_importances,
)

# Section 1: Train and Configure the Trainee

## Step 1: Load Data

We are using the Adult dataset in this recipe.

In [2]:
# Load adult data
df = fetch_data('adult', local_cache_dir="../../../data/adult")

# Specify column names
df.columns = ['age', 'workclass', 'fnlwgt', 'education',
              'education-num', 'marital-status', 'occupation',
              'relationship', 'race', 'sex', 'capital-gain',
              'capital-loss', 'hours-per-week', 'native-country', 'target']

# Sample the data for demo purpose
df = df.sample(1_000).reset_index(drop=True)

df

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,target
0,34.0,4,155343.0,11,9.0,2,3,0,4,1,0.0,1848.0,50.0,39,0
1,34.0,7,287908.0,11,9.0,4,8,3,2,1,0.0,0.0,42.0,39,1
2,27.0,0,214695.0,9,13.0,2,0,0,4,1,0.0,0.0,30.0,39,1
3,36.0,2,305714.0,11,9.0,4,8,4,2,0,0.0,0.0,70.0,39,1
4,59.0,6,156797.0,11,9.0,6,4,1,4,0,13550.0,0.0,60.0,39,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,44.0,1,399155.0,11,9.0,4,3,3,2,0,0.0,0.0,40.0,39,1
996,34.0,4,357145.0,9,13.0,2,10,5,4,0,0.0,0.0,50.0,39,0
997,36.0,4,644278.0,11,9.0,2,3,0,4,1,0.0,0.0,25.0,39,1
998,27.0,7,192355.0,9,13.0,4,10,1,4,1,0.0,0.0,50.0,39,1


## Step 2: Define the Feature Attributes

For all Howso Engine usage, we must define feature attributes. We will also define our action and context features.

In [3]:
types = {
    "education": "nominal"
}

# Infer features attributes
features = infer_feature_attributes(df, types=types)

# Specify the context and action feature
action_features = ['target']
context_features = features.get_names(without=['target'])

## Step 3: Create the Trainee, Train, and Analyze

In [4]:
# Create the trainee with custom name
t = engine.Trainee(name='Engine - Predictions and Explanations Recipe', features=features, overwrite_existing=True)

# Train
t.train(df)

# Analyze the model
t.analyze(action_features=action_features, context_features=features.get_names(without=action_features))


## Step 4: Evaluate Trainee Accuracy

In [5]:
accuracy = t.react_aggregate(
    prediction_stats_action_feature=action_features[0],
    details = {
        "prediction_stats": True,
        "selected_prediction_stats": ["accuracy"]
        }
).to_dataframe()['target'].iloc[0]

print("Test set prediction accuracy: {acc}".format(acc=accuracy))

Test set prediction accuracy: 0.82


# Section 2: Explain Predictions

How were the predictions made? 

Howso Engine provides detailed explanation for complete model transparency. Let's examine a subset of the explanations.


## Step 1: Inspect Feature Importance (Global)

Feature importance information provides insight into the features which are primary drivers for each prediction. This is important to understand in the context of AI bias and discrimination (ex. Sensitive attribute being the primary contribution to a prediction). 

This information is available at the global level (overall model), but can also be computed at the local level (regional model for each case).

In [6]:
robust_accuracy_contributions = t.react_aggregate(
    feature_influences_action_feature=action_features[0],
    details = {"feature_robust_accuracy_contributions": True}
)['feature_robust_accuracy_contributions']
plot_feature_importances(robust_accuracy_contributions.T, title="Global Mean Decrease in Accuracy (MDA)", yaxis_title="MDA")

MDA values for each feature indicate to the user which features have the most significant individual predictive power. Those with higher values are more important for the model's predictions. Furthermore, features with negative MDA values can even be considered detrimental to model accuracy. However, it's important to note that these are global values and features with negative MDA values could still be beneficial for model accuracy in some regions of the data.

## Step 2: Feature Uncertainty (Global)

Are there any noisy features? 

Howso Engine’s performance is robust against noisy feature[s], and can maintain a high level of accuracy despite noisy data. 

Part of the reason Howso Engine can maintain its level of performance despite noisy data is through the characterization of feature uncertainties (residuals). The feature residuals can be extracted for user review. 

> Note: Feature Residuals are in the same units as the original features which makes it easy to interpret. For example, the residual for the “age” feature has the unit of years as in the original data.

Feature residuals are available at the global level (overall model) and at the local level (regional model for each case).


In [7]:
global_feature_residuals = t.react_aggregate(
    prediction_stats_action_feature=action_features[0],
    details = {
        "prediction_stats": True,
        "selected_prediction_stats": ["mae"]
    }
).to_dataframe()
global_feature_residuals = global_feature_residuals.T.rename(columns={'mae':'residuals'}).sort_values('residuals', ascending=False)

global_feature_residuals.iloc[0:10]

,residuals
fnlwgt,86811.542326
capital-gain,1895.663058
capital-loss,124.054875
age,10.772250
hours-per-week,8.730351
education-num,1.445177
occupation,0.847858
workclass,0.487855
relationship,0.463170
marital-status,0.452303


# Section 3: Inspect the Data for Anomalies

Howso Engine can be used to show interesting information pertaining to the data and model, such as anomalous cases and potential model improvements. 
 
For each prediction, Howso Engine can also extract the influential cases and boundary cases to provide an exact explanation to the prediction process. More details on what’s available can be found in the notebook “2-interpretability.ipynb”.


## Step 1: Identify Anomalous cases

Anomalous cases can exist in the data as either an outlier or inlier. Outliers are cases which are very different than other cases. Inliers are cases which are too similar to other cases and do not follow the expected distribution. An inlier could be a fraudulent case that is “too good to be true”. 



In [8]:
# Store the familiarity conviction into each case with `react_into_features`,
# this will be used to identify anomalous cases
t.analyze()
t.react_into_features(familiarity_conviction_addition=True, distance_contribution=True)
stored_convictions = t.get_cases(session=t.active_session, features=df.columns.tolist() + ['familiarity_conviction_addition','.session_training_index', '.session', 'distance_contribution'])

stored_convictions

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,target,familiarity_conviction_addition,.session_training_index,.session,distance_contribution
0,34.0,4,155343.0,11,9.0,2,3,0,4,1,0.0,1848.0,50.0,39,0,0.201293,0,1fa29d55-96d0-4a65-a0bc-b98a590fd8c9,67.927966
1,34.0,7,287908.0,11,9.0,4,8,3,2,1,0.0,0.0,42.0,39,1,5.273729,1,1fa29d55-96d0-4a65-a0bc-b98a590fd8c9,12.210035
2,27.0,0,214695.0,9,13.0,2,0,0,4,1,0.0,0.0,30.0,39,1,6.593606,2,1fa29d55-96d0-4a65-a0bc-b98a590fd8c9,22.499219
3,36.0,2,305714.0,11,9.0,4,8,4,2,0,0.0,0.0,70.0,39,1,3196.508989,3,1fa29d55-96d0-4a65-a0bc-b98a590fd8c9,17.513227
4,59.0,6,156797.0,11,9.0,6,4,1,4,0,13550.0,0.0,60.0,39,0,2.568864,4,1fa29d55-96d0-4a65-a0bc-b98a590fd8c9,27.908920
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,44.0,1,399155.0,11,9.0,4,3,3,2,0,0.0,0.0,40.0,39,1,4.044713,995,1fa29d55-96d0-4a65-a0bc-b98a590fd8c9,11.536342
996,34.0,4,357145.0,9,13.0,2,10,5,4,0,0.0,0.0,50.0,39,0,4.750156,996,1fa29d55-96d0-4a65-a0bc-b98a590fd8c9,12.183666
997,36.0,4,644278.0,11,9.0,2,3,0,4,1,0.0,0.0,25.0,39,1,1.240530,997,1fa29d55-96d0-4a65-a0bc-b98a590fd8c9,30.981458
998,27.0,7,192355.0,9,13.0,4,10,1,4,1,0.0,0.0,50.0,39,1,0.672326,998,1fa29d55-96d0-4a65-a0bc-b98a590fd8c9,5.449489


Here we set a threshold for familarity conviction that can be used to identify cases as anomalies. In the case of this notebook, we use a threshold of 0.50. Cases with a conviction value below this threshold will be deemed anomalous. Then we split up these anomalies by the mean distance contribution with those lower distance contributions being inliers and those with larger distance contributions being outliers.

In [9]:
# Threshold to determine which cases will be deemed anomalous
convict_threshold = 0.5

# Extract the anomalous cases
low_convicts = stored_convictions[stored_convictions['familiarity_conviction_addition'] <= convict_threshold ].sort_values('familiarity_conviction_addition', ascending=True)

# Average distance contribution will be used to determine if a case is an outlier or inlier
average_dist_contribution = low_convicts['distance_contribution'].mean()

# A case with distance contribution greater than average will be tagged as outlier, and vise versa for inliers
cat = ['inlier' if d < average_dist_contribution else 'outlier' for d in low_convicts['distance_contribution']]
low_convicts['category'] = cat

## Step 2: Inspect Outliers

Let’s examine a few outlier cases. Outliers are cases which are very different than other cases.

In [10]:
# Extract the outliers cases
outliers = low_convicts[low_convicts['category'] == 'outlier'].reset_index(drop=True)
outliers

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,target,familiarity_conviction_addition,.session_training_index,.session,distance_contribution,category
0,69.0,4,124930.0,4,3.0,2,7,0,4,1,0.0,2267.0,40.0,39,1,0.047538,164,1fa29d55-96d0-4a65-a0bc-b98a590fd8c9,177.256596,outlier
1,59.0,4,81929.0,10,16.0,2,10,0,4,1,0.0,2415.0,45.0,39,0,0.051238,175,1fa29d55-96d0-4a65-a0bc-b98a590fd8c9,169.479005,outlier
2,42.0,4,186916.0,12,14.0,2,4,0,4,1,0.0,2415.0,55.0,39,0,0.063843,768,1fa29d55-96d0-4a65-a0bc-b98a590fd8c9,141.839272,outlier
3,48.0,6,138069.0,4,3.0,2,3,0,4,1,0.0,2042.0,50.0,39,1,0.067150,47,1fa29d55-96d0-4a65-a0bc-b98a590fd8c9,135.817078,outlier
4,33.0,4,198660.0,11,9.0,2,7,0,4,1,99999.0,0.0,56.0,39,0,0.069891,983,1fa29d55-96d0-4a65-a0bc-b98a590fd8c9,132.799296,outlier
5,51.0,4,252903.0,0,6.0,2,12,0,4,1,0.0,1977.0,40.0,39,0,0.085033,581,1fa29d55-96d0-4a65-a0bc-b98a590fd8c9,117.389788,outlier
6,57.0,5,249072.0,15,10.0,2,4,0,4,1,99999.0,0.0,60.0,39,0,0.090949,146,1fa29d55-96d0-4a65-a0bc-b98a590fd8c9,109.906306,outlier
7,37.0,4,222450.0,11,9.0,4,12,1,4,1,0.0,2339.0,40.0,8,1,0.093610,924,1fa29d55-96d0-4a65-a0bc-b98a590fd8c9,111.192498,outlier
8,46.0,4,74895.0,7,12.0,2,3,0,4,1,0.0,1485.0,55.0,39,1,0.098251,244,1fa29d55-96d0-4a65-a0bc-b98a590fd8c9,108.089849,outlier
9,42.0,4,83411.0,15,10.0,0,4,1,4,1,0.0,1408.0,40.0,39,1,0.099121,195,1fa29d55-96d0-4a65-a0bc-b98a590fd8c9,108.294409,outlier


For each outlier, we will compute some of its Feature Residual Convictions to understand which feature values are the most surprising, which will help the user understand what makes each case anomalous.

In [11]:
# Get the case_feature_residual_convictions, influential_cases and boundary_cases
details = {
    'feature_full_residual_convictions_for_case': True
}

# Specify outlier cases
outliers_indices = outliers[['.session', '.session_training_index']].values

# React to get the details of each case
results = t.react(case_indices=outliers_indices,
                  preserve_feature_values=df.columns.tolist(),
                  leave_case_out=True,
                  details=details)

In [12]:
# Extract the case feature residual convictions
case_feature_residual_convictions = results['details']['feature_full_residual_convictions_for_case'][df.columns.tolist()]

In [13]:
fig = plot_anomalies(outliers, case_feature_residual_convictions, title="Outliers", yaxis_title="Residual Conviction")
fig.show(width=1750, height=750)

The heat map explains the reason why each case was an outlier. The darker the shade of red, the more surprising the feature value is, which contributes to the case's status as an outlier.

## Step 3: Inspect Inliers

Let’s examine a few inlier cases. Inliers are cases which are too similar to other cases and do not follow the expected distribution. Inliers can be an indication of a fraudulent case that is “too good to be true”. 

In [14]:
# Get the inlier cases
inliers = low_convicts[low_convicts['category'] == 'inlier'].reset_index(drop=True)
inliers

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,target,familiarity_conviction_addition,.session_training_index,.session,distance_contribution,category
0,32.0,4,42485.0,11,9.0,0,3,4,4,1,0.0,0.0,40.0,39,1,0.184939,732,1fa29d55-96d0-4a65-a0bc-b98a590fd8c9,1.180512,inlier
1,37.0,5,175360.0,11,9.0,2,3,0,4,1,0.0,0.0,40.0,39,0,0.213102,765,1fa29d55-96d0-4a65-a0bc-b98a590fd8c9,1.522218,inlier
2,33.0,1,188246.0,11,9.0,2,3,0,4,1,0.0,0.0,40.0,39,1,0.218498,82,1fa29d55-96d0-4a65-a0bc-b98a590fd8c9,1.587135,inlier
3,34.0,4,181651.0,11,9.0,0,3,1,4,1,0.0,0.0,40.0,39,1,0.226780,963,1fa29d55-96d0-4a65-a0bc-b98a590fd8c9,1.690938,inlier
4,60.0,0,174073.0,11,9.0,2,0,0,4,1,0.0,0.0,40.0,39,1,0.244007,756,1fa29d55-96d0-4a65-a0bc-b98a590fd8c9,3.269366,inlier
5,61.0,0,175032.0,11,9.0,2,0,0,4,1,0.0,0.0,40.0,39,1,0.247591,352,1fa29d55-96d0-4a65-a0bc-b98a590fd8c9,3.309472,inlier
6,38.0,4,105503.0,15,10.0,0,4,1,4,0,0.0,0.0,40.0,39,1,0.259661,531,1fa29d55-96d0-4a65-a0bc-b98a590fd8c9,2.290774,inlier
7,24.0,4,198259.0,9,13.0,4,4,3,4,0,0.0,0.0,40.0,39,1,0.261153,818,1fa29d55-96d0-4a65-a0bc-b98a590fd8c9,2.404704,inlier
8,23.0,4,194096.0,9,13.0,4,4,3,4,0,0.0,0.0,40.0,39,1,0.262710,831,1fa29d55-96d0-4a65-a0bc-b98a590fd8c9,2.417693,inlier
9,43.0,4,150528.0,12,14.0,2,10,0,4,1,0.0,0.0,40.0,39,0,0.265063,123,1fa29d55-96d0-4a65-a0bc-b98a590fd8c9,2.751538,inlier


Similarly to what we did with the outliers, we will use the Feature Residual Convictions to understand what feature values are unusual. In this case, we may expect to see many feature values with very high convictions, indicating that the feature value is particularly unsurprising.

In [15]:
# Specify the inlier cases
inliers_indices = inliers[['.session', '.session_training_index']].values

# React to get the details of each case
results = t.react(case_indices=inliers_indices,
                  preserve_feature_values=df.columns.tolist(),
                  leave_case_out=True,
                  details=details)

In [16]:
# Extract the case feature residual convictions
case_feature_residual_convictions = results['details']['feature_full_residual_convictions_for_case'][df.columns.tolist()]

In [17]:
fig = plot_anomalies(inliers, case_feature_residual_convictions, title="Inliers", yaxis_title="Residual Conviction")
fig.show(width=1500, height=750)

The heat map explains the reason why each case was an inlier. The more unspurprising the value is. If this seems unintuitive, imagine a dataset where values are always hard to predict with an error of less than 15.0. If a case were to suddenly appear where every value was predicted perfectly, its Feature Residual Conviction values would all be extremely high (dark blue in the case of this visualization).

# Section 4: Finding Potential Improvements in the Dataset

Sparse regions of the model or under defined problems can make it difficult to make an accurate prediction. Howso Engine can be used to identify potential data, or model improvements by examining the residual conviction and density.

## Step 1: Compute Case Feature Residual Convictions

Case Feature Residual Convicion is a ratio of the local residual for each feature to the case residual. Inspecting these conviction values helps users understand what regions of their data are easier or more difficult to predict compared to the average across the dataset.

Understanding this can help users understand where more data collection may be beneficial.

In [18]:
# Identify cases for investigation
partial_train_df = stored_convictions
partial_train_cases = partial_train_df[['.session', '.session_training_index']]

In [19]:
# Residual convictions are output via the case_feature_residual_convictions explanation
details = {
    'feature_full_residual_convictions_for_case':True,
    'features': ['target'],
}

# Get the residual convictions for the specified cases
new_result = t.react(case_indices=partial_train_cases.values.tolist(),
                     leave_case_out=True,
                     preserve_feature_values=df.drop(action_features, axis=1).columns.tolist(),
                     action_features=action_features,
                     details=details)

In [20]:
# Extract residual conviction
target_residual_convictions = new_result['details']['feature_full_residual_convictions_for_case']['target']

# Binarize residual conviction
convict_threshold = 0.50
low_residual_conviction = [1 if x <= convict_threshold else 0 for x in target_residual_convictions]

# Density is just the inverse of distance_contribution
density = 1 / partial_train_df['distance_contribution']

# Add new features to the dataframe
partial_train_df['density'] = density
partial_train_df['target_residual_conviction'] = target_residual_convictions
partial_train_df['low_residual_conviction'] = low_residual_conviction

In [21]:
partial_train_df

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,...,hours-per-week,native-country,target,familiarity_conviction_addition,.session_training_index,.session,distance_contribution,density,target_residual_conviction,low_residual_conviction
0,34.0,4,155343.0,11,9.0,2,3,0,4,1,...,50.0,39,0,0.201293,0,1fa29d55-96d0-4a65-a0bc-b98a590fd8c9,67.927966,0.014721,0.428814,1
1,34.0,7,287908.0,11,9.0,4,8,3,2,1,...,42.0,39,1,5.273729,1,1fa29d55-96d0-4a65-a0bc-b98a590fd8c9,12.210035,0.081900,1.411010,0
2,27.0,0,214695.0,9,13.0,2,0,0,4,1,...,30.0,39,1,6.593606,2,1fa29d55-96d0-4a65-a0bc-b98a590fd8c9,22.499219,0.044446,0.628063,0
3,36.0,2,305714.0,11,9.0,4,8,4,2,0,...,70.0,39,1,3196.508989,3,1fa29d55-96d0-4a65-a0bc-b98a590fd8c9,17.513227,0.057100,1.235781,0
4,59.0,6,156797.0,11,9.0,6,4,1,4,0,...,60.0,39,0,2.568864,4,1fa29d55-96d0-4a65-a0bc-b98a590fd8c9,27.908920,0.035831,1.000000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,44.0,1,399155.0,11,9.0,4,3,3,2,0,...,40.0,39,1,4.044713,995,1fa29d55-96d0-4a65-a0bc-b98a590fd8c9,11.536342,0.086683,1.251994,0
996,34.0,4,357145.0,9,13.0,2,10,5,4,0,...,50.0,39,0,4.750156,996,1fa29d55-96d0-4a65-a0bc-b98a590fd8c9,12.183666,0.082077,1.586863,0
997,36.0,4,644278.0,11,9.0,2,3,0,4,1,...,25.0,39,1,1.240530,997,1fa29d55-96d0-4a65-a0bc-b98a590fd8c9,30.981458,0.032277,1.411343,0
998,27.0,7,192355.0,9,13.0,4,10,1,4,1,...,50.0,39,1,0.672326,998,1fa29d55-96d0-4a65-a0bc-b98a590fd8c9,5.449489,0.183503,1.016837,0


In [22]:
# Helper function to resize the data points
def get_sizes(min_size, max_size, series):
    min_value = series.min()
    max_value = series.max()

    m = (max_size - min_size) / (max_value - min_value)

    sizes = series * m + min_size
    return (sizes)

partial_train_df["density"] = get_sizes(5, 500, partial_train_df["density"])

With the Residual Convictions computed for the "target" feature across all the trained data, we can visualize the data to attempt to find the regions of the data where convictions are particularly low. 

In [23]:
plot_dataset(partial_train_df, x="age", y="education-num", size="density", hue="low_residual_conviction", alpha=0.4)

The above graph is a visualization of the data set in 2-dimensions, with the color as an indication of residual conviction and the size representing the density of the data. More specifically, the orange color represents the low conviction points (points which are very uncertain), and small size represents low density. Therefore, adding more data to the region with small, orange points can improve model performance. 


On the other hand, an orange point that is large would be an indication that this case lies in an dense region but was not predictable. Hence, this will be an indication where the problem is not well defined, or the data is missing key features.  


# Conclusion

In this recipe, we demonstrate many of the metrics that the Trainee provides which helps users understand the model's decision making process such as feature MDA and feature residuals.

Additionally, we show how different types of conviction can be used to identify inliers, outliers, and regions of the data where increased data collection or feature engineering may be appropriate.